In [1]:
using LinearAlgebra
using Distributions
#using ITensors
using Flux

In [2]:
struct Attention
    d_in::Int
    d_k::Int
    d_out::Int
    sqrt_d_k::Float32
    
    Wq::Matrix{Float32}
    Wk::Matrix{Float32}
    Wv::Matrix{Float32}
    bv::Vector{Float32}
end

In [3]:
Flux.trainable(a::Attention) = (Wq=a.Wq, Wk = a.Wk, Wv= a.Wv, b=a.bv)

In [4]:
function Attention(d_in::Int, d_k::Int, d_out::Int)
    sqrt_d = Float32(sqrt(d_k))
    Wq = randn(Float32, d_out, d_in)
    Wk = randn(Float32, d_k, d_in)
    Wv = randn(Float32, d_k, d_in)
    bv = randn(Float32, d_out)
    return Attention(d_in, d_k, d_out, sqrt_d, Wq, Wk, Wv, bv)
end

Attention

In [5]:
function (m::Attention)(x)
    q = m.Wq * x
    k = m.Wk * x
    v = m.Wv * x
    a = softmax(q * k' / m.sqrt_d_k, dims=2)
    return a * v + m.bv
end

In [6]:
Flux.@functor Attention

In [ ]:
test1 = randn(Float32, 5)
test2 = [randn(Float32,6,5) for i in 1:7]
test3 = [randn(Float32, 6,5) for i in 1:7]

7-element Vector{Matrix{Float32}}:
 [-0.81409454 -0.20869383 … 0.97193164 -2.116902; 1.0530068 0.6544649 … -1.636066 -0.56070244; … ; -1.0812228 -0.49389446 … -0.06371839 -1.3631718; 1.5539159 -0.32199883 … 0.2599229 -0.736534]
 [-1.0325631 -0.18857294 … 0.32509178 0.6926288; 0.08949864 -0.10854644 … 0.2191172 0.9210896; … ; -0.15776132 -0.55092055 … -0.21505569 1.2282106; -0.5913926 1.4247007 … -0.27862957 0.75134563]
 [2.0266287 -0.64107025 … -0.98688066 -0.052034505; 0.28730786 -0.17326182 … -1.1404461 0.644749; … ; -1.7576004 0.026248481 … -0.047584686 0.7500819; 1.2539276 -0.4387397 … -1.2132261 0.87527317]
 [-0.9561609 -0.44869602 … 0.69290215 1.3344662; 1.6643621 -0.006622632 … 1.0215122 0.89319646; … ; -1.3223414 -0.92579544 … 0.017907474 -0.82925045; 0.98250103 0.3117501 … -0.7721408 0.58566725]
 [-0.97074735 0.4353392 … -0.92750955 0.38294294; -1.0954165 -0.6006523 … 0.35420522 -1.1531465; … ; 0.093750164 -0.8665578 … 2.2179585 0.120194025; -1.7994345 0.91248685 … 0.3707132 -

In [7]:
model = Chain(
    Dense(10, 10),
    Attention(10, 10, 10),
    Dense(10, 3)
)

Chain(
  Dense(10 => 10),                      # 110 parameters
  Attention(10, 10, 10, 3.1622777f0, Float32[2.0596619 -1.2176862 … 1.3844051 0.2670034; -1.5038713 -0.6465389 … -0.116612665 0.6202918; … ; 1.9936777 -2.3028915 … -0.71147734 0.31237927; 0.53021574 0.6561479 … -1.29335 -0.40964413], Float32[1.198084 1.325444 … -1.1987529 -1.6271359; 0.2862526 -1.4459498 … 0.11082824 -0.08883958; … ; 1.7614883 -1.3242586 … 0.19767357 0.97850734; -0.51289207 0.37245914 … 0.31699434 0.36281878], Float32[-1.5879103 -1.1571153 … 0.7843065 0.27988526; -0.3934454 0.040846 … -0.18600781 -0.9443482; … ; -1.3232015 0.6443577 … 0.50792396 0.96673214; 0.38206506 -1.7438259 … -1.7665398 -0.46390042], Float32[0.3978311, -1.0406922, 1.3473414, 1.5462812, -0.12485075, -0.16762353, 1.8197639, 0.05612002, -0.08261416, 0.43578944]),  # 310 parameters
  Dense(10 => 3),                       # 33 parameters
)                   # Total: 8 arrays, 453 parameters, 2.363 KiB.

In [8]:
test_in = randn(Float32, 10)

10-element Vector{Float32}:
 -1.3780282
  0.51189923
 -1.6344907
 -1.6006424
  0.255674
  0.9674617
  1.3480097
  2.0480268
 -0.65438485
 -1.7909698

In [9]:
model(test_in)

3-element Vector{Float32}:
  2.078571
 -0.009895287
  2.2800577

In [15]:
test_data = [(randn(Float32, 10), randn(Float32, 3)) for i in 1:1000]

loss(x, y) = Flux.mse(model(x), y)
opt = ADAM()

Flux.train!(loss, Flux.params(model), test_data, opt)

In [18]:
test_data[1][2]

3-element Vector{Float32}:
  0.76775545
 -0.69179815
 -1.0929852

In [19]:
model(test_data[1][1])

3-element Vector{Float32}:
 -0.42953014
 -0.6347547
  0.54480594

In [10]:
struct RMSLayerNorm
    d_in::Int
    sqrt_d::Float32
    g::Vector{Float32}
end

In [11]:
Flux.trainable(a::RMSLayerNorm) = (g=a.g)

In [12]:
function RMSLayerNorm(d_in::Int)
    g = ones(Float32, d_in)
    sqrt_d = Float32(sqrt(d_in))
    return RMSLayerNorm(d_in, sqrt_d, g)
end

RMSLayerNorm

In [13]:
function (m::RMSLayerNorm)(x)
    return (m.sqrt_d * x) ./ sqrt(x'*x) .* m.g
end

In [14]:
Flux.@functor RMSLayerNorm

In [15]:
model = Chain(
    Dense(10, 10),
    Attention(10, 10, 10),
    RMSLayerNorm(10),
    Dense(10, 3)
)

Chain(
  Dense(10 => 10),                      # 110 parameters
  Attention(10, 10, 10, 3.1622777f0, Float32[0.5895154 -0.45373893 … 1.1242785 0.22795276; 0.2307629 0.06996048 … 0.43961883 -0.31826496; … ; 0.27553117 -0.050983854 … -1.9729972 0.6427319; 0.88272595 -0.17197134 … 0.5263166 0.2811659], Float32[-0.8154785 1.0239226 … -0.06907645 2.281529; -0.15978236 0.7747966 … -0.043165367 1.3398274; … ; -0.95428485 0.315605 … 0.62446606 -1.0768476; 1.0024437 -1.4333708 … 0.53480244 -0.68439925], Float32[-1.4448279 -0.35359675 … 1.4133186 -0.47379982; -0.07821751 0.56915224 … 0.0022872176 0.6309256; … ; 1.4897128 1.982514 … 1.4500712 -1.2206903; 0.91974485 -0.36394265 … -0.6727435 -1.7026688], Float32[1.1686106, -1.0038536, -1.4668595, 0.74383765, -0.06287023, -1.8724244, -0.6357962, 1.0010865, 0.8288817, 0.35795978]),  # 310 parameters
  RMSLayerNorm(10, 3.1622777f0, Float32[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]),
  Dense(10 => 3),                       # 33 parameters
)    

In [16]:
model(test_in)

3-element Vector{Float32}:
 -0.534494
  0.44231528
 -1.0940514